<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/data_cleaning/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skript um die Messdaten un Vorhersagedaten zusammen zu fühern und dieses zu bereinigen. Zudem werden weitere Features die für das Trainig des Netzwerks wichtig sind erzeugt.

Dateiname: Data_Cleaning

[Notebook 0: Data Cleaning](./0_DataCleaning.ipynb)

[Notebook 1: EDA](./1_EDA_and_Cleaning.ipynb)

[Notebook 2: Modeling and Predictions](./2_Modeling_and_Predictions.ipynb)

[Notebook 3: Technical Report](./3_Technical_Report.ipynb)

## Installation der Bibliothek PVlib:
Dieses Modul wird für die Berechnung des Sonnenstandwinkels verwendet.

In [1]:
# Installation der pvlib um den Sonnenstandswinkel zu berechnen
!pip install pvlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 24.4 MB/s eta 0:00:00


## Import der benötigten Module und allgemeines Setup:

In [2]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import der benötigten Module

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pvlib

from datetime import datetime
from datetime import timedelta

import matplotlib
import seaborn as sns

In [4]:
# Standort der Messstaion für die Berechnung des Sonnenstandswinkel
latitude = 50.8
longitude = 7.2

# Angaben um das Datum und die Uhrzeit in Sin und Cos zu codieren
seconds_in_day = 24*60*60
seconds_in_year = (365.2425)*seconds_in_day

In [5]:
# Pfad zu den Messwerten / Vorhersagewerten und dem Speicherort auf Google Drive
# Import
folder_UVI = '/content/drive/My Drive/Colab_Notebooks/CSV_UVI/'
folder_Solys = '/content/drive/My Drive/Colab_Notebooks/SOLYS_CSV/'
folder_CAMS = '/content/drive/My Drive/Colab_Notebooks/CAMS_Vorhersage/'
folder_VarIdx = '/content/drive/My Drive/Colab_Notebooks/CAMS_Vorhersage/'

name_UVI = ['22.06', '22.07', '22.08', '22.09', '22.10', '22.11', '22.12', '23.01', '23.02', '23.03', '23.04', '23.05', '23.07', '23.08']  # Hier wird angegeben, welche Monate importiert werden sollen
name_Solys = 'Solys_CSV'
name_CAMS = 'CAMS_std_CSV'
name_CAMS_Glo = 'CAMS_Glo_CSV'
name_CAMS_TCC = 'CAMS_TCC_CSV'
name_VarIdx = 'blabla'

# Export

## Import der UVI-Messdaten:

Die Messdaten sind in CSV-Dateien gespeichert und müssen importiert werden.

In [30]:
## Code zum Import der Messdaten
dataframes = []
df_UVI_combined = []

for name in name_UVI:
  file_path = folder_UVI + name
  # print(file_path)
  df_import = pd.read_csv(file_path)
  dataframes.append(df_import)

df_UVI_combined = pd.concat(dataframes, ignore_index=True)
df_UVI_combined['Datetime'] = pd.to_datetime(df_UVI_combined['Datetime'])
df_UVI_combined.set_index('Datetime', inplace = True)
df_UVI_combined.drop(['Datum', 'Uhrzeit', 'Messzeitpunkt'], axis = 1)

,UVI,UVA,UVB,erythem
Datetime,,,,
2022-06-15 07:21:00,2.408378,4.686417,281.330695,0.060209
2022-06-15 07:23:00,2.462381,4.793073,287.094062,0.061560
2022-06-15 07:25:00,2.479048,4.817792,288.902613,0.061976
2022-06-15 07:27:00,2.543531,4.953151,292.450776,0.063588
2022-06-15 07:29:00,2.576485,5.016957,295.552724,0.064412
...,...,...,...,...
2023-08-12 18:34:00,0.024202,NaN,NaN,0.000605
2023-08-12 18:36:00,0.020477,NaN,NaN,0.000512
2023-08-12 18:38:00,0.019234,NaN,NaN,0.000481


In [31]:
df_UVI_combined

,UVI,UVA,UVB,erythem,Datum,Uhrzeit,Messzeitpunkt
Datetime,,,,,,,
2022-06-15 07:21:00,2.408378,4.686417,281.330695,0.060209,2022-06-15,07:21:00,26460
2022-06-15 07:23:00,2.462381,4.793073,287.094062,0.061560,2022-06-15,07:23:00,26580
2022-06-15 07:25:00,2.479048,4.817792,288.902613,0.061976,2022-06-15,07:25:00,26700
2022-06-15 07:27:00,2.543531,4.953151,292.450776,0.063588,2022-06-15,07:27:00,26820
2022-06-15 07:29:00,2.576485,5.016957,295.552724,0.064412,2022-06-15,07:29:00,26940
...,...,...,...,...,...,...,...
2023-08-12 18:34:00,0.024202,NaN,NaN,0.000605,2023-08-12,18:34:00,66840
2023-08-12 18:36:00,0.020477,NaN,NaN,0.000512,2023-08-12,18:36:00,66960
2023-08-12 18:38:00,0.019234,NaN,NaN,0.000481,2023-08-12,18:38:00,67080


In [7]:
# df_UVI_combined.fillna(0)

## Import der Solys-Messdaten:

Messdaten mit einer 2-minütigen Auflösung. Die Daten sind noch nicht reduziert.

In [8]:
# Einlesen der Solys-Messdaten:
df_Solys = pd.read_csv(folder_Solys + name_Solys)

# Die Spalte mit dem Zeitstempel in das Datetimeformat umwandeln und als Index speichern
df_Solys['Datetime'] = pd.to_datetime(df_Solys['Datetime'])
df_Solys.set_index('Datetime', inplace = True)

In [26]:
df_Solys

,Glo,Dif,Glo_SPLite,Dir,Temp
Datetime,,,,,
2022-04-25 08:06:00,245.833333,246.866667,259.233333,2.400000,10.766667
2022-04-25 08:08:00,237.058333,237.666667,249.200000,2.350000,10.658333
2022-04-25 08:10:00,206.466667,206.675000,217.633333,2.316667,10.058333
2022-04-25 08:12:00,188.858333,189.400000,200.900000,2.316667,10.608333
2022-04-25 08:14:00,186.991667,187.616667,199.800000,2.208333,10.391667
...,...,...,...,...,...
2023-11-09 08:10:00,9.975000,10.500000,11.266667,1.400000,8.683333
2023-11-09 08:12:00,10.233333,10.741667,11.291667,1.400000,8.708333
2023-11-09 08:14:00,10.775000,11.275000,11.691667,1.416667,8.716667


## Import der CAMS-Vorhersagedaten:

Die Daten sind noch nicht reduziert und es gibt für jede Stunde des Tages eine Vorhersage.

In [10]:
# Einlesen der Solys-Messdaten:
df_CAMS = pd.read_csv(folder_CAMS + name_CAMS)

# Die Spalte mit dem Zeitstempel in das Datetimeformat umwandeln und als Index speichern
df_CAMS['Datetime'] = pd.to_datetime(df_CAMS['Datetime'])
df_CAMS.set_index('Datetime', inplace = True)

In [25]:
df_CAMS

,aod469,aod550,gtco3,uvbed,uvbedcs,hcc,lcc,tcc
Datetime,,,,,,,,
2022-06-01 00:00:00,0.261836,0.211588,0.008161,-5.551115e-16,5.551115e-16,0.000000,0.000000,0.504647
2022-06-01 01:00:00,0.272371,0.220283,0.008044,-5.551115e-16,5.551115e-16,0.000000,0.000000,0.681122
2022-06-01 02:00:00,0.237297,0.190401,0.007992,-5.551115e-16,5.551115e-16,0.124487,0.015687,0.414249
2022-06-01 03:00:00,0.195020,0.155880,0.007971,-5.551115e-16,5.551115e-16,0.071994,0.005310,0.076343
2022-06-01 04:00:00,0.164975,0.131777,0.007959,7.204596e-02,7.198133e-02,0.000000,0.001114,0.001785
...,...,...,...,...,...,...,...,...
2023-07-31 19:00:00,0.131174,0.109557,0.006663,5.551115e-16,5.551115e-16,1.000000,0.259121,1.000000
2023-07-31 20:00:00,0.126791,0.106162,0.006666,5.551115e-16,5.551115e-16,1.000000,0.473365,1.000000
2023-07-31 21:00:00,0.133466,0.111809,0.006700,5.551115e-16,5.551115e-16,0.999878,0.934537,1.000000


## Import der CAMS-Globalstrahlungs-Vorhersagedaten:

In [12]:
# Einlesen der Solys-Messdaten:
df_CAMS_GLO = pd.read_csv(folder_CAMS + name_CAMS_Glo)

# Die Spalte mit dem Zeitstempel in das Datetimeformat umwandeln und als Index speichern
df_CAMS_GLO['Datetime'] = pd.to_datetime(df_CAMS_GLO['Observation_period'])
df_CAMS_GLO.set_index('Datetime', inplace = True)
df_CAMS_GLO.drop(['Unnamed: 0', 'Observation_period'], axis = 1)

,Clear_sky_GHI,Clear_sky_BHI,GHI,BHI
Datetime,,,,
2022-06-01 00:01:00,0.0,0.0,0.0,0.0
2022-06-01 00:03:00,0.0,0.0,0.0,0.0
2022-06-01 00:05:00,0.0,0.0,0.0,0.0
2022-06-01 00:07:00,0.0,0.0,0.0,0.0
2022-06-01 00:09:00,0.0,0.0,0.0,0.0
...,...,...,...,...
2023-08-31 23:51:00,0.0,0.0,0.0,0.0
2023-08-31 23:53:00,0.0,0.0,0.0,0.0
2023-08-31 23:55:00,0.0,0.0,0.0,0.0


## Import der CAMS-TCC-Vorhersagedaten:

In [13]:
# Einlesen der Solys-Messdaten:
df_CAMS_TCC = pd.read_csv(folder_CAMS + name_CAMS_TCC, index_col=[0, 1])

# ! Der Index muss noch in Datetime umgewndelt werden !

# Die Spalte mit dem Zeitstempel in das Datetimeformat umwandeln und als Index speichern
# df_CAMS_TCC['Datetime'] = pd.to_datetime(df_CAMS_TCC['Datetime'])
# df_CAMS_TCC.set_index('Datetime', inplace = True)

In [14]:
# df_CAMS_TCC

## Zusammenführen der DataFrames zu einem gemeinsamen DataFrame

In [71]:
def merge_asof_multiple_dfs(dfs, tolerance=pd.Timedelta('1 minute')):

    # Initialisiere das finale DataFrame mit dem ersten DataFrame in der Liste
    final_df = dfs[0].sort_values('Datetime')

    # Iteriere über die restlichen DataFrames in der Liste und führe sie schrittweise zusammen
    for df in dfs[1:]:
        df = df.sort_values('Datetime')
        final_df = pd.merge_asof(final_df, df, on='Datetime', tolerance=tolerance, direction='nearest')

    return final_df

In [72]:
dataframes = [df_CAMS_GLO, df_UVI_combined, df_Solys]  # df_CAMS_TCC

df_complete = []
df_complete = merge_asof_multiple_dfs(dataframes)

In [73]:
df_complete

,Datetime,Unnamed: 0,Observation_period,Clear_sky_GHI,Clear_sky_BHI,GHI,BHI,UVI,UVA,UVB,erythem,Datum,Uhrzeit,Messzeitpunkt,Glo,Dif,Glo_SPLite,Dir,Temp
0,2022-06-01 00:01:00,0,2022-06-01 00:01:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.275000,10.600
1,2022-06-01 00:03:00,1,2022-06-01 00:03:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.266667,10.575
2,2022-06-01 00:05:00,2,2022-06-01 00:05:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.300000,10.550
3,2022-06-01 00:07:00,3,2022-06-01 00:07:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.283333,10.525
4,2022-06-01 00:09:00,4,2022-06-01 00:09:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.225000,10.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329035,2023-08-31 23:51:00,329035,2023-08-31 23:51:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.516667,13.900
329036,2023-08-31 23:53:00,329036,2023-08-31 23:53:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.583333,13.900
329037,2023-08-31 23:55:00,329037,2023-08-31 23:55:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.600000,13.900
329038,2023-08-31 23:57:00,329038,2023-08-31 23:57:00,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.600000,13.900


In [37]:
print(df_UVI_combined, df_Solys, df_CAMS_GLO)

                          UVI       UVA         UVB   erythem       Datum  \
Datetime                                                                    
2022-06-15 07:21:00  2.408378  4.686417  281.330695  0.060209  2022-06-15   
2022-06-15 07:23:00  2.462381  4.793073  287.094062  0.061560  2022-06-15   
2022-06-15 07:25:00  2.479048  4.817792  288.902613  0.061976  2022-06-15   
2022-06-15 07:27:00  2.543531  4.953151  292.450776  0.063588  2022-06-15   
2022-06-15 07:29:00  2.576485  5.016957  295.552724  0.064412  2022-06-15   
...                       ...       ...         ...       ...         ...   
2023-08-12 18:34:00  0.024202       NaN         NaN  0.000605  2023-08-12   
2023-08-12 18:36:00  0.020477       NaN         NaN  0.000512  2023-08-12   
2023-08-12 18:38:00  0.019234       NaN         NaN  0.000481  2023-08-12   
2023-08-12 18:40:00  0.018458       NaN         NaN  0.000461  2023-08-12   
2023-08-12 18:42:00  0.014863       NaN         NaN  0.000372  2023-08-12   

## Bereinigen der Messdaten

- Fehlende Messtage müssen ersetzt werden:
  - Prüfen, ob die UVI-Messwerte zusammenhängen.
  - Hinzufügen neuer Zeilen, falls die Messwerte nicht zusammen Hängen.

**Hier wird geprüft, ob die Messwerte zusammenhängend sind:**

In [19]:
def insert_missing_rows(df):
    # Sortieren des DataFrame nach 'Datetime'
    df.sort_values(by='Datetime', inplace=True)

    # Initialisieren einer Liste, um die Zeilen mit fehlenden Daten einzufügen
    rows_to_insert = []

    # Gruppieren des DataFrame nach 'Datum'
    grouped = df.groupby('Datum')

    for date, group in grouped:
        # Sortieren der Gruppe nach 'Datetime'
        group.sort_values(by='Datetime', inplace=True)

        for i in range(1, len(group)):
            current_time = group.iloc[i]['Datetime']
            prev_time = group.iloc[i - 1]['Datetime']
            time_diff = current_time - prev_time

            if time_diff > timedelta(minutes=2):
                while prev_time + timedelta(minutes=2) < current_time:
                    prev_time += timedelta(minutes=2)
                    new_row = {
                        'Datetime': prev_time,
                        'Datum': date,
                        'Uhrzeit': prev_time.time(),
                        'Messzeitpunkt': (prev_time - prev_time.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds(),
                        'erythem': 0,
                        'UVI': 0,
                        'DiffGreater2': 1,
                    }
                    rows_to_insert.append(new_row)

    # Einfügen der fehlenden Zeilen in einen DataFrame
    if rows_to_insert:
        new_rows_df = pd.DataFrame(rows_to_insert)
        df = pd.concat([df, new_rows_df], ignore_index=True)

    # Sortieren des DataFrame nach 'Datetime'
    df.sort_values(by='Datetime', inplace=True)
    df = df.reset_index(drop = True)
    df['DiffGreater2'] = df['DiffGreater2'].fillna(0)

    return df

In [20]:
df_UVI_WRows = pd.DataFrame()
df_UVI_WRows = insert_missing_rows(df_UVI_combined)

In [21]:
# df_UVI_WRows

,Datetime,UVI,UVA,UVB,erythem,Datum,Uhrzeit,Messzeitpunkt,DiffGreater2
0,2022-06-15 07:21:00,2.408378,4.686417,281.330695,0.060209,2022-06-15,07:21:00,26460.0,0.0
1,2022-06-15 07:23:00,2.462381,4.793073,287.094062,0.061560,2022-06-15,07:23:00,26580.0,0.0
2,2022-06-15 07:25:00,2.479048,4.817792,288.902613,0.061976,2022-06-15,07:25:00,26700.0,0.0
3,2022-06-15 07:27:00,2.543531,4.953151,292.450776,0.063588,2022-06-15,07:27:00,26820.0,0.0
4,2022-06-15 07:29:00,2.576485,5.016957,295.552724,0.064412,2022-06-15,07:29:00,26940.0,0.0
...,...,...,...,...,...,...,...,...,...
120023,2023-08-19 05:48:00,0.306965,NaN,NaN,0.007674,2023-08-19,05:48:00,20880.0,0.0
120024,2023-08-19 05:50:00,0.312439,NaN,NaN,0.007811,2023-08-19,05:50:00,21000.0,0.0
120025,2023-08-19 05:52:00,0.314239,NaN,NaN,0.007856,2023-08-19,05:52:00,21120.0,0.0
120026,2023-08-19 05:54:00,0.316836,NaN,NaN,0.007921,2023-08-19,05:54:00,21240.0,0.0


## Zussammenführen der Daten und löschen nicht benötigter Einträge:

## Sonnenstandswinkel hinzufügen

In [22]:
def calculate_solar_zenith_angle(dataframe, date_column, latitude, longitude, altitude=0):

    # Kopieren des ursprünglichen DataFrames.
    result_df = dataframe.copy()

    # Konvertiere der Datumsspalte in einen datetime-Datentyp, falls sie dies nicht bereits ist.
    if not pd.api.types.is_datetime64_any_dtype(dataframe[date_column]):
        result_df[date_column] = pd.to_datetime(dataframe[date_column])

    # Iteriere über die Zeilen des DataFrames und berechne den Solarzenitwinkel für jedes Datum.
    solar_zenith_angles = []
    for date in result_df[date_column]:
        solar_position = pvlib.solarposition.get_solarposition(date, latitude, longitude, altitude)
        solar_zenith_angle = solar_position['zenith'].values[0]
        solar_zenith_angles.append(solar_zenith_angle)

    # Füge die berechneten Solarzenitwinkel dem DataFrame hinzu.
    result_df['SZA'] = solar_zenith_angles

    return result_df

In [23]:
df_UVI_WRows_SZ = pd.DataFrame()
df_UVI_WRows_SZ = calculate_solar_zenith_angle(df_UVI_WRows, 'Datetime', latitude, longitude)

## Zeit und Datum in Sin und Cos codieren
- Cyclical Encoding der Messzeit als Sinus und Cosinus

In [ ]:
def calculate_date_in_sine_cosine(dataframe, day, year):

    # kopieren des ursprünglichen DataFrames
    result_df = dataframe.copy()

    # berechnen der Uhrzeit und des Datums als Sin und Cos
    result_df['time_sin'] = np.sin(2*np.pi*result_df['Messzeitpunkt']/day)
    result_df['time_cos'] = np.cos(2*np.pi*result_df['Messzeitpunkt']/day)
    result_df['date_sin'] = np.sin((2*np.pi*result_df['Datetime'].dt.dayofyear * 24 * 60 * 60) / year)
    result_df['date_cos'] = np.cos((2*np.pi*result_df['Datetime'].dt.dayofyear * 24 * 60 * 60) / year)

    return result_df

In [ ]:
# Erweitern des DataFrames mit den UVI-Messwerten um den Solarzenitwinkel und der Uhrzeit als Sin/Cos
df_UVI_SZ_SC = calculate_date_in_sine_cosine(df_UVI_WRows_SZ, seconds_in_day, seconds_in_year)

## Erstellen einer Liste mit Stunden an denen es Messwerte gibt:

In [ ]:
# Herausschreiben des Datums und der Stunde aus den Messdaten um im Folgeden nur die relevanten Vorhersagedaten auszuwählen
df_UVI_SZ_SC['Date'] = df_UVI_SZ_SC['Datetime'].dt.date
df_UVI_SZ_SC['Hour'] = df_UVI_SZ_SC['Datetime'].dt.hour
#
df_date_std = df_UVI_SZ_SC.groupby(['Date', 'Hour']).size().reset_index(name='Count')
# DateTime-Splate mit dem Messzeitpunkt hinzufügen
df_date_std['Datetime'] = pd.to_datetime(df_date_std['Date'].astype(str) + ' ' + df_date_std['Hour'].astype(str) + ':00:00')
# DateTime-Spalte als Index setzen
df_date_std.set_index('Datetime', inplace=True)

In [24]:
# df_date_std

NameError: name 'df_date_std' is not defined

## Erstellen einer Liste mit Zeitstemplen an denen es Messwerte gibt:

In [ ]:
df_date = pd.DataFrame(pd.to_datetime(df_UVI_SZ_SC['Datetime']))
df_date.set_index('Datetime', inplace=True)

## Import der Vorhersagedaten und löschen nicht gebrauchter Einträge:

In [ ]:
# Datetime als Index festlegen
df_CAMS.set_index('Datetime', inplace=True)

In [ ]:
# Löschen aller nicht relevanten Einträge
df_cams_std = df_CAMS[df_CAMS.index.isin(df_date_std.index)]

## Löschen nicht gebrauchter Solys Strahlungsdaten
Die Strahlungsdaten werden an die UVI-Messdaten angepasst

In [ ]:
df_Solys_std = pd.merge_asof(df_date, df_Solys, left_index=True, right_index=True, tolerance=pd.Timedelta('1min'))

In [ ]:
df_Solys_std

## Abspeichern des DataFrames als CSV

In [ ]:
df_UVI_SZ_SC.isnull().sum()

In [ ]:
# Irradiance metrics over time
df_UVI_SZ_SC[['UVI']].plot()
plt.title('UVI über die Zeit');

In [ ]:
def export_dataframes_to_csv(df1, df2, df3, file1_name, file2_name, file3_name, folder_name):

    try:
        # Exportiere den ersten DataFrame in eine CSV-Datei
        df1.to_csv(folder_name + file1_name)
        print(f'DataFrame 1 wurde erfolgreich in "{file1_name}" exportiert.')

        # Exportiere den zweiten DataFrame in eine CSV-Datei
        df2.to_csv(folder_name + file2_name)
        print(f'DataFrame 2 wurde erfolgreich in "{file2_name}" exportiert.')

        # Exportiere den dritten DataFrame in eine CSV-Datei
        df3.to_csv(folder_name + file3_name)
        print(f'DataFrame 3 wurde erfolgreich in "{file3_name}" exportiert.')

    except Exception as e:
        print(f'Fehler beim Export der DataFrames: {str(e)}')

In [ ]:
export_dataframes_to_csv(df_UVI_SZ_SC, df_Solys_std, df_cams_std, 'UVI_Messdaten.csv', 'Solys_Messdaten.csv', 'Vorhersage.csv', save_folder)